# Importing Relevant Libraries

In [129]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from scipy import stats

#### Loading the Data

In [131]:
file_path = "Files/2.02.+Binary+predictors.csv"
df = pd.read_csv(file_path)
copy_df = df.copy()

In [133]:
copy_df.Admitted = copy_df.Admitted.map({"Yes":1, "No":0})
copy_df.Gender = copy_df.Gender.map({"Female":1, "Male":0})
copy_df

,SAT,Admitted,Gender
0,1363,0,0
1,1792,1,1
2,1954,1,1
3,1653,0,0
4,1593,0,0
...,...,...,...
163,1722,1,1
164,1750,1,0
165,1555,0,0
166,1524,0,0


In [135]:
y = copy_df.Admitted
x1 = copy_df.Gender

#### Regression

In [138]:
x = sm.add_constant(x1)
reg_log = sm.Logit(y,x)
results_log = reg_log.fit()
results_log.summary()

Optimization terminated successfully.
         Current function value: 0.572260
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Admitted   No. Observations:                  168
Model:                          Logit   Df Residuals:                      166
Method:                           MLE   Df Model:                            1
Date:                Sat, 13 Jul 2024   Pseudo R-squ.:                  0.1659
Time:                        17:01:22   Log-Likelihood:                -96.140
converged:                       True   LL-Null:                       -115.26
Covariance Type:            nonrobust   LLR p-value:                 6.283e-10
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.6436      0.222     -2.901      0.004      -1.078      -0.209
Gender         2.0786      0.363      5.727      0.000       1.367       2.790
==============================================================================
"""

#### Interpreting Results

#### Running another regression inclding SAT and Gender

In [142]:
file_path = "Files/2.02.+Binary+predictors.csv"
df = pd.read_csv(file_path)
copy_df = df.copy()

In [144]:
copy_df.Admitted = copy_df.Admitted.map({"Yes":1, "No":0})
copy_df.Gender = copy_df.Gender.map({"Female":1, "Male":0})
copy_df

,SAT,Admitted,Gender
0,1363,0,0
1,1792,1,1
2,1954,1,1
3,1653,0,0
4,1593,0,0
...,...,...,...
163,1722,1,1
164,1750,1,0
165,1555,0,0
166,1524,0,0


In [146]:
y = copy_df["Admitted"]
x1 = copy_df[["SAT","Gender"]]

In [148]:
x = sm.add_constant(x1)
reg_log = sm.Logit(y,x)
results_log = reg_log.fit()
results_log.summary()

Optimization terminated successfully.
         Current function value: 0.120117
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Admitted   No. Observations:                  168
Model:                          Logit   Df Residuals:                      165
Method:                           MLE   Df Model:                            2
Date:                Sat, 13 Jul 2024   Pseudo R-squ.:                  0.8249
Time:                        17:01:30   Log-Likelihood:                -20.180
converged:                       True   LL-Null:                       -115.26
Covariance Type:            nonrobust   LLR p-value:                 5.118e-42
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -68.3489     16.454     -4.154      0.000    -100.598     -36.100
SAT            0.0406      0.010      4.129      0.000       0.021       0.060
Gender         1.9449      0.846      2.299      0.022       0.287       3.603
==============================================================================

Possibly complete quasi-separation: A fraction 0.27 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

#### Interpretation

#### Creating the Prediction table output and interpretation

In [155]:
confution_m = pd.DataFrame(results_log.pred_table())
confution_m.columns = ["Predicted 0", "Predicted 1"]
confution_m = confution_m.rename(index={0:"Actual 0",1:"Actual 1"})
confution_m

,Predicted 0,Predicted 1
Actual 0,69.0,5.0
Actual 1,4.0,90.0


#### Testing the Model (split test)

In [158]:
# Making the input data same as inital input and loading the split data
test_data = pd.read_csv("Files/2.03.+Test+dataset.csv")
test_data.head()

,SAT,Admitted,Gender
0,1323,No,Male
1,1725,Yes,Female
2,1762,Yes,Female
3,1777,Yes,Male
4,1665,No,Male


In [160]:
test_data["Admitted"] = test_data["Admitted"].map({"Yes":1,"No":0})
test_data["Gender"] = test_data["Gender"].map({"Female":1,"Male":0})
test_data.head()

,SAT,Admitted,Gender
0,1323,0,0
1,1725,1,1
2,1762,1,1
3,1777,1,0
4,1665,0,0


In [164]:
test_result = test_data["Admitted"]
test = test_data.drop(["Admitted"],axis = 1)
test = sm.add_constant(test)
test.head()

,const,SAT,Gender
0,1.0,1323,0
1,1.0,1725,1
2,1.0,1762,1
3,1.0,1777,0
4,1.0,1665,0


#### Creating the confution matrix with test data

In [167]:
def confution_matrix (data,actual_values,model):
    pred_values = model.predict(data)
    bins = np.array([0,0.5,1])
    c_matrix = np.histogram2d(actual_values,pred_values,bins = bins)[0]
    accuracy = (c_matrix[0,0]+c_matrix[1,1])/c_matrix.sum()
    return c_matrix, accuracy

In [169]:
confution_matrix(test,test_result,results_log)

(array([[5.00, 1.00],
        [1.00, 12.00]]),
 0.8947368421052632)

#### Interpretation